In [1]:
import cv2
from keras.utils import Sequence
import numpy as np
import pandas as pd
import zipfile
import sys
sys.path.append('../')
from model.model import vgg16Model

c:\users\vaibhav\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
c:\users\vaibhav\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\vaibhav\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("qu

In [6]:
#Skip files not present
class My_Custom_Generator_Images(Sequence) :
    def __init__(self,image_filenames, batch_size,type_of_data, archive) :
        self.image_filenames = image_filenames
        self.batch_size = batch_size
        self.type_of_data = type_of_data
        self.archive = archive
        
    def __len__(self) :
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
    
    def __getitem__(self, idx) :
        batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        images_x = []
        for counter, img in enumerate(batch_x):
            try:
                zeros = 12 - len(str(img))
                if(self.type_of_data=='val'):
                    file_name = 'COCO_val2014_' + '0'*zeros + str(img) + '.jpg'
                    imgData = self.archive.read('val2014/' + file_name)
                    im = cv2.imdecode(np.frombuffer(imgData, np.uint8), 1)
                if(self.type_of_data=='train'):
                    file_name = 'COCO_train2014_' + '0'*zeros + str(img) + '.jpg'
                    imgData = self.archive.read('train2014/' + file_name)
                    im = cv2.imdecode(np.frombuffer(imgData, np.uint8), 1)
                im = cv2.resize(im, (224,224))
                im = im/255.0
                images_x.append(im)
            except:
                print(counter)
                continue
        return np.asarray(images_x)

In [ ]:
def getImageFeatures(dataframe_path, images_path, column_name, type_of_data, model,batch_size=128):
    df = pd.read_csv(dataframe_path)
    archive = zipfile.ZipFile(images_path, 'r')
    unique_values = df[column_name].unique()
    generator = My_Custom_Generator_Images(unique_values, batch_size, type_of_data, archive)
    predictions = model.predict_generator(generator, verbose = 1)
    return unique_values, predictions

In [ ]:
vgg16 = vgg16Model(1000)

In [ ]:
values, predictions = getImageFeatures('../dataset/train_final.csv', '../../Dataset-Coco-V2/vqa_v2_images.zip', 'image_id', 'train', vgg16, 128)
train_features = {}
for i in range(len(values)):
    train_features[values[i]] = predictions[i]
values, predictions = getImageFeatures('../dataset/val_final.csv', '../../Dataset-Coco-V2/vqa_v2_validation_images.zip', 'image_id', 'val', vgg16, 128)
val_features = {}
for i in range(len(values)):
    val_features[values[i]] = predictions[i]